In [1]:
import glob
import os
import time

from selenium import webdriver

In [2]:
from Config import params_config

In [3]:
parameters = params_config.parameters

In [4]:
parameters

{'TARGET_URL': 'https://race.netkeiba.com/?pid=schedule&select=schedule&year={YEAR}',
 'YEAR_RANGE': range(2002, 2020),
 'DRIVER_MAC': './chromedriver',
 'PAGE_LOAD_TIMEOUT': 10}

In [5]:
driver = webdriver.Chrome(executable_path=parameters['DRIVER_MAC'])
driver.set_page_load_timeout(parameters['PAGE_LOAD_TIMEOUT'])
driver.maximize_window()

In [6]:
year = parameters['YEAR_RANGE'][17]
year

2019

In [7]:
target_url = parameters['TARGET_URL'].format(YEAR=year)
target_url

'https://race.netkeiba.com/?pid=schedule&select=schedule&year=2019'

In [8]:
try:
    driver.get(target_url)
except:
    print("The page load was time out")